In [1]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import transect_viz
from transect_viz import transect_cdec_data
from transect_viz import transect_generator
import hvplot.pandas

In [20]:
df=transect_viz.load_transect_file('../data/Transect_20220302_Fabian.csv')
df['EC'] = df['Sonde1_SpCond'] # 'EC' is mapped from Sonde1_SpCond
df.head(3)

,DateTime,RECORD,RunName,Longitude,Latitude,GPS_SOG,Lag Estimate (ft),GPS_COG,PumpVolt,Sonde1_Temp,...,Sonde2_Temp,Sonde2_SpCond,Sonde2_Turbidity,Sonde2_DO_sat,Sonde2_DO_conc,Sonde2_Depth,Sonde2_pH,Sonde2_Chlor_ugL,Pump,EC
0,3/2/2022 10:52,41358,Run_01,-121.457512,37.802799,0.0,0.0,0.0,11.64388,13.431,...,13.042,842.51,2.09,95.20,10.00,1.353,7.71,3.50,On,840.80
1,3/2/2022 10:52,41359,Run_01,-121.457512,37.802799,0.0,0.0,0.0,11.64569,13.429,...,13.041,842.52,2.16,95.15,9.99,1.353,7.70,3.23,On,841.13
2,3/2/2022 10:52,41360,Run_01,-121.457520,37.802799,0.0,0.0,0.0,11.62740,13.427,...,13.040,842.52,2.18,95.12,9.99,1.357,7.70,3.17,On,841.11


In [21]:
import panel as pn
pn.extension()

import holoviews as hv

In [22]:
def map_with_legend_by_color_size(df, cmap='rainbow4'):
    mapcz, legend = transect_viz.map_transect_with_size_and_color(df, cmap=cmap)
    return mapcz, legend

In [23]:
#list(hv.element.tile_sources.keys()) # full_list
tiles=pn.widgets.Select(options=['CartoLight','StamenTerrain','OSM','EsriImagery'], value='CartoLight')

In [24]:
#from holoviews.plotting import list_cmaps
#list_cmaps(category='Uniform Sequential')
#list_cmaps()

cmap_list=['rainbow4','inferno','viridis','ColorBlind']
cmaps=pn.widgets.Select(options=cmap_list, value='rainbow4')

In [25]:
#mapcs = pn.Column(pn.Column(tiles,cmaps), pn.bind(map_by_color_size, df=df, tile_layer=tiles, cmap=cmaps))
#mapcs 

In [26]:
cdec_stations=['ORM', 'ODM', 'OLD', 'GLE', 'GLC']
station_info_file='../data/station_info_file.csv'

In [27]:
dfs = transect_cdec_data.get_stations_cached(cdec_stations)
dfs = transect_viz.add_in_station_info(dfs, station_info_file=station_info_file)

In [28]:
def create_transect_profile(df):
    gdf = transect_generator.to_geoframe(df)
    transect_line = transect_generator.create_transect_line(gdf)
    gdf = transect_generator.add_transect_dist(gdf, transect_line)
    return gdf.hvplot.scatter(x='transect_dist', y='EC').opts(
        title='Profile of Specific Conductivity along Transect ', xlabel='Transect Distance (ft)', ylabel='Specific Conductivity')

In [29]:
transect_profile_plot = create_transect_profile(df)
transect_profile_plot

:Scatter   [transect_dist]   (EC)

In [30]:
station_map_with_tiles = transect_viz.get_tile_layer('CartoLight') *\
    transect_viz.points_for_station(dfs) *\
    transect_viz.labels_for_stations(dfs).opts(frame_height=350)

In [31]:
sdate, edate = transect_viz.get_start_end_dates(df)
bsdate, bedate = transect_viz.get_buffered_start_end_dates(df)

In [32]:
def make_list(df):
    return [df[[c]] for c in df.columns]

In [33]:
cdec_flows = transect_cdec_data.get_cdec_data_cached(bsdate, bedate, cdec_stations, data_type='flow')
cdec_flows = make_list(cdec_flows)

In [34]:
cdec_vels = transect_cdec_data.get_cdec_data_cached(bsdate, bedate, cdec_stations, data_type='vel')
cdec_vels = make_list(cdec_vels)

In [35]:
collection_span = transect_viz.timespan(sdate, edate)

In [36]:
flow_plot = transect_viz.plot_flow_around_time(sdate,edate,cdec_flows)*collection_span

In [37]:
velocity_plot = transect_viz.plot_velocity_around_time(sdate,edate,cdec_vels)*collection_span

In [38]:
from holoviews import opts

In [39]:
dftidalvelgis = transect_viz.merge_mean_velocity_with_stations(dfs, transect_viz.mean_velocity_over_time(sdate,edate,transect_viz.get_tidal_filtered(cdec_vels)))

In [40]:
vel_tidal_vectors = transect_viz.plot_velocity_vectors(dftidalvelgis)*transect_viz.plot_velocity_labels(dftidalvelgis)

In [41]:
vel_tidal_map_with_station = station_map_with_tiles*vel_tidal_vectors
vel_tidal_map_with_station.opts(frame_height=350, title='Mean Tidally filtered Velocity over collection period', xlabel='Longitude', ylabel='Latitude')

:Overlay
   .Tiles.I       :Tiles   [x,y]
   .Points.I      :Points   [Longitude,Latitude]   (Station ID)
   .Labels.I      :Labels   [Longitude,Latitude]   (Station ID)
   .VectorField.I :VectorField   [Longitude,Latitude]   (angle,mean velocity)
   .Labels.II     :Labels   [Longitude,Latitude]   (label)

In [42]:
dfvelgis = transect_viz.merge_mean_velocity_with_stations(dfs, transect_viz.mean_velocity_over_time(sdate,edate,cdec_vels))

In [43]:
vel_vectors = transect_viz.plot_velocity_vectors(dfvelgis)*transect_viz.plot_velocity_labels(dfvelgis)

In [44]:
#ymin, ymax = hv.util.transform.lon_lat_to_easting_northing(-121.5,37.78)[1], hv.util.transform.lon_lat_to_easting_northing(-121.5,37.83)[1]
#ylim=(ymin,ymax))

In [45]:
vel_map_with_station = station_map_with_tiles*vel_vectors
vel_map_with_station.opts(frame_height=350, title='Mean Velocity over collection period', xlabel='Longitude', ylabel='Latitude')

:Overlay
   .Tiles.I       :Tiles   [x,y]
   .Points.I      :Points   [Longitude,Latitude]   (Station ID)
   .Labels.I      :Labels   [Longitude,Latitude]   (Station ID)
   .VectorField.I :VectorField   [Longitude,Latitude]   (angle,mean velocity)
   .Labels.II     :Labels   [Longitude,Latitude]   (label)

In [46]:
def show_begin_end_pts_with_labels(df):
    begin_end = df.iloc[[0, -1]][['Longitude', 'Latitude', 'DateTime']].copy()
    begin_end['Label'] = ['Start', 'End']
    print(begin_end)
    plt = begin_end.iloc[0:1, :].hvplot.labels(
        x='Longitude', y='Latitude', text='Label', geo=True).opts(opts.Labels(yoffset=300))
    plt = plt * begin_end.iloc[-1:, :].hvplot.labels(
        x='Longitude', y='Latitude', text='Label', geo=True).opts(opts.Labels(yoffset=-300))
    plt = plt * begin_end.hvplot.points(geo=True)
    return plt

In [47]:
vel_map_with_station*show_begin_end_pts_with_labels(df)

       Longitude   Latitude        DateTime  Label
0    -121.457512  37.802799  3/2/2022 10:52  Start
6012 -121.457512  37.802795  3/2/2022 13:01    End


:Overlay
   .Tiles.I       :Tiles   [x,y]
   .Points.I      :Points   [Longitude,Latitude]   (Station ID)
   .Labels.I      :Labels   [Longitude,Latitude]   (Station ID)
   .VectorField.I :VectorField   [Longitude,Latitude]   (angle,mean velocity)
   .Labels.II     :Labels   [Longitude,Latitude]   (label)
   .Labels.III    :Labels   [Longitude,Latitude]   (Label)
   .Labels.IV     :Labels   [Longitude,Latitude]   (Label)
   .Points.II     :Points   [Longitude,Latitude]

In [48]:
map1, legend1 = map_with_legend_by_color_size(df)
map1 = transect_viz.get_tile_layer('CartoLight')*map1.opts(xlabel='Longitude', ylabel='Latitude')
map1 = map1.opts(opts.Points(colorbar=True)).opts(title='Transect EC by size and color')

In [49]:
vel_map2=pn.Row(vel_map_with_station.opts(frame_width=700, frame_height=300), vel_tidal_map_with_station.opts(frame_width=700, frame_height=300))

In [50]:
#map_panel = pn.Row(pn.Column(map1.opts(frame_width=700, frame_height=300),vel_map2),legend1.opts(title='Point Size Legend'))
map_panel = pn.Row(pn.Column(pn.Row(map1.opts(frame_width=700, frame_height=300),legend1.opts(title='Point Size Legend')),
                             pn.Row(vel_map_with_station.opts(frame_width=700, frame_height=300),
                                   transect_profile_plot)))

In [51]:
#list(hv.element.tile_sources.keys()) # full_list
tiles=pn.widgets.Select(options=['CartoLight','StamenTerrain','OSM','EsriImagery'], value='CartoLight')

In [52]:
#from holoviews.plotting import list_cmaps
#list_cmaps(category='Uniform Sequential')
#list_cmaps()

cmap_list=['rainbow4','inferno','viridis','ColorBlind']
cmaps=pn.widgets.Select(options=cmap_list, value='rainbow4')

In [53]:
#mapcs = pn.Column(pn.Column(tiles,cmaps), pn.bind(map_by_color_size, df=df, tile_layer=tiles, cmap=cmaps))
#mapcs 

In [54]:
tidal_plot = transect_viz.plot_tidal_filtered_flow_around_time(sdate, edate, transect_viz.get_tidal_filtered(cdec_flows))*collection_span

In [55]:
tidal_vel_plot = transect_viz.plot_tidal_filtered_velocity_around_time(sdate, edate, transect_viz.get_tidal_filtered(cdec_vels))*collection_span

In [56]:
flow_velocity_panel = pn.Column(pn.Row(flow_plot, tidal_plot), pn.Row(velocity_plot, tidal_vel_plot))

In [57]:
cdec_stage = transect_cdec_data.get_cdec_data_cached(
    bsdate, bedate, cdec_stations, data_type='stage')
cdec_stage = make_list(cdec_stage)
cdec_ec = transect_cdec_data.get_cdec_data_cached(
    bsdate, bedate, cdec_stations, data_type='ec')
cdec_ec = make_list(cdec_ec)

stage_plot = transect_viz.plot_data_around_time(
    sdate, edate, cdec_stage, data_type='stage', ylabel='Stage (ft)', title='Stage')*transect_viz.timespan(sdate, edate)

tidal_stage_plot = transect_viz.plot_data_around_time(sdate, edate, transect_viz.get_tidal_filtered(
    cdec_stage), data_type='stage', ylabel='Stage (ft)', title='Stage tidally filtered')*transect_viz.timespan(sdate, edate)

ec_plot = transect_viz.plot_data_around_time(
    sdate, edate, cdec_ec, data_type='ec', ylabel='EC (umhos/cm)', title='EC')*transect_viz.timespan(sdate, edate)

tidal_ec_plot = transect_viz.plot_data_around_time(sdate, edate, transect_viz.get_tidal_filtered(
    cdec_ec), data_type='ec', ylabel='Tidally filtered EC (umhos/cm)', title='EC')*transect_viz.timespan(sdate, edate)

In [58]:
stage_ec_panel = pn.Column(pn.Row(stage_plot, tidal_stage_plot), pn.Row(ec_plot, tidal_ec_plot))

In [59]:
full_panel = pn.Column(map_panel, flow_velocity_panel, stage_ec_panel)

In [60]:
full_panel

Column
    [0] Row
        [0] Column
            [0] Row
                [0] HoloViews(Overlay)
                [1] HoloViews(Overlay)
            [1] Row
                [0] HoloViews(Overlay)
                [1] HoloViews(Scatter)
    [1] Column
        [0] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
    [2] Column
        [0] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)
        [1] Row
            [0] HoloViews(Overlay)
            [1] HoloViews(Overlay)

In [62]:
pn.io.save.save(full_panel,'../transect_reports/Transect_Viz_20220302_Fabian.html',title='Fabian Tract 2022-03-02 EC Transect Visualization Panel')